In [6]:
import pandas as pd
import numpy as np
import warnings
import re
import math
# warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

chem_list = [
    u'C',
    u'Mn',
    u'Si',
    u'P',
    u'S',
    u'Cr',
    u'Ni',
    u'Cu',
    u'Al',
    u'V',
    u'Ti',
    u'Nb',
    u'Mo',
    u'N',
    u'B'
]

def ch2f(x):
    if (type(x)==float) | (type(x)==int):
        x = str(x)
    if u'г' in x:
        print(x)
        x = x.replace(u'г.п.', '')
        x = x.replace(u'гп', '')
        x = str(x)
    if u'0.012. 0.014. 0.013' in str(x):
        x = x.replace(u'0.012. 0.014. 0.013', u'0.013')
    if x[-4:-2]=='e-':
        return float(x)
    elif '-' in str(x): 
#         print(x)
        x = x.replace(' ', '')
        x = str(x) + '0'
        tmp_map = list(map(float, str(x).replace(',', '.').split('-')))
        if (abs(tmp_map[-1]-tmp_map[-2])>0.05) | (tmp_map[-1]==0):
            return np.mean(tmp_map[:-1])
        else:
            return np.mean(tmp_map)
    else:
        return float(str(x).replace(',', '.'))


# подгрузка всех файлов испытаний
import os
dr = 'DATA/июнь 20-26/июнь испытания 2018 yu.xls'

# файлы в словарь доступ по имени файла ({имя файла: словарь датафреймов файла} )
exp_files = pd.read_excel(dr, None)
#     print(name)

In [9]:

# номера плавок приводим к единому регистру (нижнему)
for l in exp_files:
    if '№ плавки' in exp_files[l]:
        exp_files[l]['№ плавки'] = exp_files[l]['№ плавки'].astype(str).str.lower()
    if '№ партии' in exp_files[l]:
        exp_files[l]['№ партии'] = exp_files[l]['№ партии'].astype(str).str.lower()

# создаем единую таблицу с химией
chemistry = []
chemistry = exp_files['Хим.анализ плавок']

In [11]:
chemistry[u'№ плавки'] = chemistry[u'№ плавки'].astype(str)

In [12]:
for el in chem_list:
    chemistry[el]=chemistry[el].apply(ch2f)

def get_more_chem(x):
    for el in x:
        for i in x[el]:
            if i!=None:
                x[el] = i
    return x
                
chemistry = chemistry.groupby('№ плавки').apply(get_more_chem)

chemistry.drop_duplicates(inplace=True)
chemistry.shape

(133, 104)

In [13]:
# подгружаем режимы
mode = pd.read_excel('DATA/prepared/valid_june.xlsx')
mode.shape
mode[u'№ плавки'] = mode[u'№ плавки'].astype(str)
mode.shape

(168, 21)

In [14]:
merge_table = pd.merge(mode, chemistry, how ='left', on=['№ плавки'])
print(merge_table.shape)

(168, 124)


In [15]:
merge_table = merge_table[merge_table['C'].notnull()]
print(merge_table.shape)

(165, 124)


In [17]:
copy_merge_table = merge_table.copy()
copy_merge_table.shape

(165, 124)

In [18]:
def clean_by_chem(df, mark, el, right_b, left_b=None):
    tmp = df.shape[0]
    if left_b==None:
        df = df[((df['марка стали']==mark) & ((df[el]<=right_b)|(pd.isnull(df[el]))))| (df['марка стали']!=mark)]
    else:
        df = df[((df['марка стали']==mark) & ((df[el]<=right_b)|(pd.isnull(df[el]))) & ((df[el]>=left_b)|(pd.isnull(df[el])))) | (df['марка стали']!=mark)]
#     print(df.shape)
    if (df.shape[0]-tmp!=0):
        print('удалено строк:', np.abs(df.shape[0]-tmp), 'причина', el, mark)
    return df

def clean_all_mark_chem(merge_table):
    # чистим 09Г2C
    merge_table = clean_by_chem(merge_table, '09Г2С', 'C', 0.14, 0.03)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Mn', 1.8, 1)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Si', 0.75, 0.45)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'S', 0.05)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'P', 0.05)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Al', 0.06, 0.015)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Ti', 0.035)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Ni', 0.3)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Cr', 0.3)
    merge_table = clean_by_chem(merge_table, '09Г2С', 'Cu', 0.3)
    # чистим 13ХФА
    merge_table = clean_by_chem(merge_table, '13ХФА', 'C', 0.19, 0.1)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Mn', 0.7, 0.3)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Si', 0.4, 0.13)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'Cr', 0.8, 0.4)
    merge_table = clean_by_chem(merge_table, '13ХФА', 'V', 0.08, 0.02)
    # чистим 30Г2
    merge_table = clean_by_chem(merge_table, '30Г2', 'C', 0.35, 0.25)
    merge_table = clean_by_chem(merge_table, '30Г2', 'Mn', 1.6, 0.7)
    merge_table = clean_by_chem(merge_table, '30Г2', 'Si', 0.5, 0.13)
    # чистим 18ХМФБ
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'C', 0.25, 0.12)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Mn', 0.7, 0.3)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Si', 0.5, 0.13)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Cr', 1.2, 0.45)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Mo', 0.4, 0.1)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'V', 0.12, 0.02)
    merge_table = clean_by_chem(merge_table, '18ХМФБ', 'Nb', 0.08, 0.015)
    # чистим 37Г2Ф
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'C', 0.45, 0.35)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'Mn', 1.8, 0.7)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'Si', 0.5, 0.13)
    merge_table = clean_by_chem(merge_table, '37Г2Ф', 'V', 0.12, 0.02)
    return merge_table

# merge_table[merge_table['№ партии']==203010]

merge_table = clean_all_mark_chem(merge_table)
merge_table.shape


merge_table.to_excel('DATA/prepared/june_merge_table_mean_chem.xlsx')

удалено строк: 1 причина Cr 18ХМФБ
удалено строк: 2 причина C 37Г2Ф
